In [ ]:
pip install seaborn

In [ ]:
# pip install nltk


In [ ]:
import os 
import pandas as pd

from sklearn.model_selection import train_test_split
import numpy as np
dir_name= os.getcwd()+'/data/'


In [ ]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import twitter_samples 

analyzer = SentimentIntensityAnalyzer()

In [ ]:
def read_tweets_us_path(path):
    print('read_us_path')
    path=os.path.join(dir_name,'train-data/'+path)
    df=pd.read_csv(path)
    cs=[]
    for row in range(len(df)):
        cs.append(analyzer.polarity_scores(df['tweets'].iloc[row])['compound'])
    df['compound_vader_score']=cs
    print(df)
    return df

def read_news_us_path(path):
    print('read_us_path')
    path=os.path.join(dir_name,'train-data/'+path)
    df=pd.read_csv(path)
    cs=[]
    for row in range(len(df)):
        cs.append(analyzer.polarity_scores(df['news'].iloc[row])['compound'])
    df['compound_vader_score']=cs
#     df = df[(df[['compound_vader_score']] != 0).all(axis=1)].reset_index(drop=True)
    print(df)
    return df

In [ ]:
def find_tweets_pred_label(grouped_data,file_name,perc_change):
    print('find_pred_label')
    tweets=grouped_data['tweets']
    grouped_data=grouped_data.groupby(['dates','ticker'])['compound_vader_score'].mean().reset_index()
    final_label=[]
    for i in range(len(grouped_data)):

        if grouped_data['compound_vader_score'].iloc[i]>perc_change:
            final_label.append(2)
        elif grouped_data['compound_vader_score'].iloc[i]<-perc_change:
            final_label.append(0)
        elif (grouped_data['compound_vader_score'].iloc[i]>=-perc_change  and grouped_data['compound_vader_score'].iloc[i]<=perc_change):
            final_label.append(1)

    grouped_data['vader_label']=final_label
    grouped_data['tweets']=tweets
    grouped_data.to_csv(file_name)
    
def find_news_pred_label(grouped_data,file_name,perc_change):
    print('find_pred_label')
    tweets=grouped_data['news']
    grouped_data=grouped_data.groupby(['dates','ticker'])['compound_vader_score'].mean().reset_index()
    final_label=[]
    for i in range(len(grouped_data)):
        if grouped_data['compound_vader_score'].iloc[i]>perc_change:
            final_label.append(2)
        elif grouped_data['compound_vader_score'].iloc[i]<-perc_change:
            final_label.append(0)
        elif (grouped_data['compound_vader_score'].iloc[i]>=-perc_change  
              and grouped_data['compound_vader_score'].iloc[i]<=perc_change):
            final_label.append(1)

    grouped_data['vader_label']=final_label
    grouped_data['news']=tweets
    grouped_data.to_csv(file_name)


In [ ]:

def merge_actual_label (join_path,label_data):
    print('merge_actual_label')
    vader_data=pd.read_csv(join_path)
    vader_data.set_index(keys = ["dates","ticker"],inplace=True)
#     print(vader_data)
    label_data=pd.read_csv(label_data)
    label_data.set_index(keys = ["dates","ticker"],inplace=True)
#     print(label_data)
    merge=pd.merge(vader_data,label_data, how='inner', left_index=True, right_index=True)
    merge.drop(columns=['Unnamed: 0_y'],axis=1)
    print(merge)
    return merge


In [ ]:
import seaborn as sns
def make_confusion_matrix(cf,group_names=None,categories='auto',count=True,percent=True,cbar=True,xyticks=True,xyplotlabels=True,
sum_stats=True,figsize=None,cmap='Blues', title=None):

    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])


    if sum_stats:
        accuracy  = np.trace(cf) / float(np.sum(cf))
        if len(cf)==2:
            precision = cf[1,1] / sum(cf[:,1])
            recall    = cf[1,1] / sum(cf[1,:])
            f1_score  = 2*precision*recall / (precision + recall)
            stats_text = "\n\nAccuracy={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={:0.3f}".format(
                accuracy,precision,recall,f1_score)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""



    if figsize==None:

        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        categories=False


    # MAKE THE HEATMAP VISUALIZATION
    plt.figure(figsize=figsize)
    sns.heatmap(cf,annot=box_labels,fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories)

    if xyplotlabels:
        plt.ylabel('True label')
        plt.xlabel('Predicted label' + stats_text)
    else:
        plt.xlabel(stats_text)
    
    if title:
        plt.title(title)

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

def validation(df,name):
    print(df)
    pred_label=list(df['vader_label'])
    actual_label=list(df['label'])
    labels=[0,1,2]
    cm=confusion_matrix(actual_label, pred_label,labels)
    labels = ['True Neg','False Pos','False Neg','True Pos']
    categories = ['Negative','Neutral', 'Positive']
    make_confusion_matrix(cm, group_names=labels, categories=categories )
    df.to_csv(name)
    
    

In [ ]:
# 
def us_tweets_starter(path,file_name,label_data,percent_change):
    grouped_data=read_tweets_us_path(path)
    find_tweets_pred_label(grouped_data,file_name,percent_change)
    df=merge_actual_label (file_name,label_data)
    return df


def us_news_starter(path,file_name,label_data,percent_change):
    grouped_data=read_news_us_path(path)
    find_news_pred_label(grouped_data,file_name,percent_change)
    df=merge_actual_label (file_name,label_data)
    
    return df
 

In [ ]:
# getting nasdaq tweets
# threshold=np.linspace(0.0,0.02, num=5)
# nasdaq_tweets_accuracies=[]
# for val in threshold:
#     print (val)


nasdaq_tweets_path='nasdaq/nasdaq_no_labelled_tweets.csv'
nasdaq_vader_tweets_path=os.path.join(dir_name,'train-data/nasdaq/nasdaq_vader_tweets.csv')
join_path=os.path.join(dir_name,'train-data/nasdaq/nasdaq_labelled_data.csv')
nasdaq_tweets_df=us_tweets_starter(nasdaq_tweets_path,nasdaq_vader_tweets_path,join_path,0.01)
# nasdaq_tweets_accuracies.append(find_accuracy(nasdaq_tweets_df))

In [ ]:

import matplotlib.pyplot as plt
def plot_accuracy(accuracies):
    x=accuracies
    y=np.linspace(0.0,0.02, num=5)
    plt.plot(y,x)
    plt.ylabel('Accuracies')
    plt.xlabel('threshold')
    
#     plt.annotate('Local Max', x =) 
    plt.show() 
    
#find_accuracy(nasdaq_tweets_df)
def find_accuracy(name):
    pred=list(name['pred_label'])
    act=list(name['label'])
    true=0
    false=0
    for i in range(len(pred)):
        if(pred[i]==act[i]):
            true+=1
        else:
            false+=1
    acc=true/len(pred)
    print(acc)
    return acc

# plot_accuracy(nasdaq_tweets_accuracies)

In [ ]:
validation(nasdaq_tweets_df,'nasdaq_vader_tweets_label.csv')

In [ ]:
# getting nasdaq news
# threshold=np.linspace(0.0, 0.02, num=5)
# nasdaq_news_accuracies=[]
# for val in threshold:
#     print(val)
nasdaq_news_path='nasdaq/nasdaq_no_labelled_news.csv'

nasdaq_vader_news_path=os.path.join(dir_name,'train-data/nasdaq/nasdaq_vader_news.csv')
join_path=os.path.join(dir_name,'train-data/nasdaq/nasdaq_labelled_data.csv')

nasdaq_news_df=us_news_starter(nasdaq_news_path,nasdaq_vader_news_path,join_path,0.01)
# nasdaq_news_accuracies.append(find_accuracy(nasdaq_news_df))


In [ ]:
validation(nasdaq_news_df,'nasdaq_vader_news_label.csv')

In [ ]:
# getting nyse news
# threshold=np.linspace(0.0, 0.02, num=5)
# nyse_newss_accuracies=[]
# for val in threshold:
#     print(val)
nyse_news_path='nyse/nyse_no_labelled_news.csv'
nyse_vader_news_path=os.path.join(dir_name,'train-data/nyse/nyse_vader_news.csv')
join_path=os.path.join(dir_name,'train-data/nyse/nyse_labelled_data.csv')
        
nyse_news_df=us_news_starter(nyse_news_path,nyse_vader_news_path,join_path,0.01)
#     nyse_newss_accuracies.append(find_accuracy(nyse_news_df))

In [ ]:
validation(nyse_news_df,'nyse_vader_news_label.csv')

In [ ]:
# getting nyse tweets

# threshold=np.linspace(0.0, 0.02, num=5)
# nyse_tweets_accuracies=[]
# for val in threshold:
#     print(val)
nyse_tweets_path='nyse/nyse_no_labelled_tweets.csv'
nyse_vader_tweets_path=os.path.join(dir_name,'train-data/nyse/nyse_vader_tweets.csv')
join_path=os.path.join(dir_name,'train-data/nyse/nyse_labelled_data.csv')
nyse_tweets_df=us_tweets_starter(nyse_tweets_path,nyse_vader_tweets_path,join_path,0.01)
#     nyse_tweets_accuracies.append(find_accuracy(nyse_tweets_df))


In [ ]:
# # validations
validation(nyse_tweets_df,'nyse_vader_tweets.csv')

In [ ]:
# nyse_news_path='nyse/nyse_no_labelled_news.csv'
# nyse_vader_news_path=os.path.join(dir_name,'train-data/nyse/nyse_vader_news.csv')
# join_path=os.path.join(dir_name,'train-data/nyse/nyse_labelled_data.csv')

# nyse_news_df=us_news_starter(nyse_news_path,nyse_vader_news_path,join_path,0.00)



# threshold=np.linspace(0.0, 0.02, num=5)
# hk_accuracies=[]
# for val in threshold:
#     print (val)
hkex_news_path='hkex/hkex_no_labelled_news.csv'
hkex_vader_news_path=os.path.join(dir_name,'train-data/hkex/hkex_vader_news.csv')
join_path=os.path.join(dir_name,'train-data/hkex/hkex_labelled_data.csv')
hkex_news_df=us_news_starter(hkex_news_path,hkex_vader_news_path,join_path,0.01)
#     hk_accuracies.append(find_accuracy(hkex_news_df))


In [ ]:
validation(hkex_news_df,'hkex_vader_news.csv')

In [ ]:
plot_accuracy(nasdaq_tweets_accuracies)
plot_accuracy(nasdaq_news_accuracies)
plot_accuracy(nyse_tweets_accuracies)
plot_accuracy(nyse_newss_accuracies)
plot_accuracy(hk_accuracies)

In [ ]:
nasdaq_vader_news_path=os.path.join(dir_name,'train-data/nasdaq/nasdaq_vader_news.csv')
df=pd.read_csv(nasdaq_vader_news_path)
df.set_index(keys = ["dates","ticker"],inplace=True)
df=df.drop('Unnamed: 0',axis=1)
print(df)